'''
v0: 국내 통합코드 적용
v1: title 전처리
v2: name 전처리
v3: 통합 안되는 친구들 필터 테스트, name 다국어 변환
v4: name 유사도 비교
v5: email 비교
'''

In [1]:
from pymongo import MongoClient
import itertools
import numpy as np
import jaro
import time
#import multicpu

client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')
filter_info = client['PUBLIC']['FilterInfo'] #필터접근
filters_category = client['PUBLIC']['FilterCategory']

f_id = 0 #input
keyid = 888 #keyid

fid_key_query = filter_info.find_one({ '$and': [{ 'fId': f_id }, { 'keyId': keyid }]}) #f_id serach

pinst = []
pyear = []
pjournal = []
plang = []

if  fid_key_query != None: #f_id check
    for key in fid_key_query.keys() :
        if key == 'pFilter':
            pinst = fid_key_query[key]['inst']
            pyear = fid_key_query[key]['year']
            pjournal = fid_key_query[key]['journal']
            plang = fid_key_query[key]['lang']

wos_raw = client['WOS']['Rawdata']
scopus_raw = client['SCOPUS']['Rawdata']

wos_key_query = wos_raw.find({ 'keyId' : keyid })
scopus_key_query = scopus_raw.find({ 'keyId' : keyid })

key_querys = [wos_key_query, scopus_key_query] #Rawdata
id_domestic = client['ID']['Domestic'] #Domestic

mng_id = [] # Author id
paper = []

Answer_dict = {} # Answer result
fp_dict = {} #filter papaer result
site = ['WOS', 'SCOPUS']

savetime1 = 0
savetime2 = 0
end1 = 0

f_pyear = {}
f_pinst = {}
f_pjournal = {}
f_plang = {}

Inte_name = []

def different_lang(name):
    # Conversion_list = ['í', 'é', 'ô', 'ń', 'ł', 'á', 'ú', 'ä']
    Conversion_list = ['i', 'e', 'o', 'n', 'l', 'a', 'u', 'a']
    Conversion_index = [237, 233, 244, 324, 322, 225, 250, 228]
    list_name = list(name)
    
    for i in range(len(list_name)):        
        if ord(list_name[i]) in Conversion_index:
            list_name[i] = Conversion_list[Conversion_index.index(ord(list_name[i]))]
            
    name = "".join(list_name)
    return name

def simple_filter(value, filters):
    if value in filters or filters == []:
        return True
    return False
        
def fc_simple_filter(category, fc_dict):
    if category not in fc_dict:
        fc_dict[category] = 0
    fc_dict[category] += 1
    return fc_dict

for i in range(len(key_querys)):
    mng_dict = {}
    start1 = time.time()
    for key_query in key_querys[i]: #rawdata(magid, paper) insert

        if key_query['author_inst'] not in "":
            paper_year =  str(key_query['issue_year'])
            paper_journal = key_query['journal']
            # ori_inst = key_query['originalName'].split(';')[-2]
            ori_inst = key_query['author_inst'].split(';')[-2]
            paper_lang = key_query['issue_lang']
            exi_inst = key_query['author_inst'].split(';')[-2]
            mng_name = key_query['author'].split(';')[-2]
            mng_id = key_query['author_id'].split(';')[-1]
            paper = key_query['_id']
            
        if simple_filter(paper_year, pyear) and simple_filter(paper_journal, pjournal) and simple_filter(ori_inst, pinst) and simple_filter(paper_lang, plang):
            if mng_id not in mng_dict:
                mng_dict[mng_id] = {'name' : mng_name, 'inst' : exi_inst, 'papers' : [], 'oriInst' : ori_inst}
            mng_dict[mng_id]['papers'].append(paper)
            fp_dict[paper] = {'year' : paper_year, 'inst' : ori_inst, 'journal' : paper_journal, 'lang' : paper_lang}
            
    end2 = time.time()
    db_time = end2-start1
    print(f'DB 수집: {site[i]}, {db_time}')
    savetime1 += db_time
    len_mng_dict = len(mng_dict)
    print(f'전체 저자 수: {len_mng_dict}')
    
    for mng_one in mng_dict :
        oriinst = mng_dict[mng_one]['oriInst']
        exiinst = mng_dict[mng_one]['inst']
        mng_name = mng_dict[mng_one]['name']
        paper = mng_dict[mng_one]['papers']
        pre_name = different_lang(mng_name.lower().replace(" ", "").replace(",", "").replace(".", "").replace("-", "").replace("´", ""))
        Answer = {'fid': f_id, 'keyId': keyid, 'name' : mng_name, 'pre_name' : pre_name, 'inst': oriinst, site[i] : {'inst' :exiinst, 'A_id': [mng_one], 'papers' : paper, 'oriInst' : oriinst} }
        
        if pre_name not in Answer_dict and pre_name+'_0' not in Answer_dict : #동명이인이 없을 때
            Answer_dict[pre_name] = Answer

        else:
            count = 0
            flag = True
            while flag :
                temp = None
                tempName = pre_name
                
                if tempName in Answer_dict : # 이름 으로만 key가ㅣ 존재         
                    temp = Answer_dict[tempName]
                    flag = False
                else :
                    tempName = pre_name+'_'+str(count) # 이름 + 숫자로 key가ㅣ 존재
                    if tempName not in Answer_dict :
                        flag = False 
                        break
                    temp = Answer_dict[tempName]
                        
                for key in temp.keys() : # 사이트 돌면서
                    if key != 'name' and key != 'keyId' and key != 'fid' and key != 'inst' and key != 'pre_name': 
                        src = ""
                        tgt = ""

                        if len(exiinst) >= len(temp[key]['inst']):
                            src = temp[key]['inst']
                            tgt = exiinst

                        elif len(exiinst) < len(temp[key]['inst']):
                            src = exiinst
                            tgt = temp[key]['inst']

                        if key == site[i] :# 사이트가 동일할때
                            if temp[key]['inst'] == exiinst or (src != "" and src in tgt) :  # 소속 같을때
                                Inte_name.append(tempName)
                                Answer_dict[tempName][site[i]]['A_id'].extend([mng_one])
                                Answer_dict[tempName][site[i]]['papers'].extend(paper)
                                flag = False
                                break

                            elif pre_name+'_'+str(count+1) not in Answer_dict : #소속이 다를 때
                                Answer_dict[pre_name+'_'+str(count+1)] = Answer
                                if tempName == pre_name:
                                    Answer_dict[pre_name+'_0'] = temp
                                    del Answer_dict[pre_name]
                                flag = False
                                break
                            
                        else :# 사이트가 다를때 
                            if temp[key]['inst'] == exiinst  or (src != "" and src in tgt):  # 소속 같을때
                                Answer_dict[tempName][site[i]] =  {'inst' : exiinst, 'A_id': [mng_one], 'papers' : paper, 'oriInst' : oriinst}
                                Inte_name.append(tempName)
                                if  Answer_dict[tempName]['inst'] == "" or Answer_dict[tempName]['inst'] == " ":
                                    Answer_dict[tempName]['inst'] = Answer_dict[tempName][site[i]]['oriInst']
                                flag = False
                                break
                            
                            elif pre_name+'_'+str(count+1) not in Answer_dict : #소속이 다를 때
                                Answer_dict[pre_name+'_'+str(count+1)] = Answer
                                if tempName == pre_name:
                                    Answer_dict[pre_name+'_0'] = temp
                                    del Answer_dict[pre_name]
                                flag = False
                                break

                count += 1
                
end3 = time.time()
savetime2 = end3-end2+savetime1

print(f'2차 통합: {savetime2}')
print(Inte_name)
print(len(Answer_dict))

DB 수집: WOS, 8.754547595977783
전체 저자 수: 20037
DB 수집: SCOPUS, 16.88533639907837
전체 저자 수: 20230
2차 통합: 25.810181140899658
['vandenbruelann', 'yasunagahideo_1', 'kilicarman_0', 'maeviv', 'wuszuyuan_2', 'tzengniansheng_2', 'mummanenipraveenv_1', 'delacruzdelrioramamaria', 'liujianping_0', 'habibiaslsaeede', 'chenweiliang_0', 'lixin_2', 'elhajjalbert', 'karakiewiczpierrei_3', 'wangxuehao_2', 'hejie_6', 'fanx_1', 'zhangpeng_3', 'huy_2', 'habibiasls', 'zhangl_7', 'sunx_2']
40245


In [4]:
def filter(rawdata, site):
    coauthor = rawdata['author'].split(";")[:-1]
    year = int(rawdata['issue_year'])
    paper_keyword = rawdata['paper_keyword']
    
    if paper_keyword == [] or paper_keyword is None:
        keyword = []
    else:
        keyword = paper_keyword.split(";")

    journal = rawdata['journal']
    conference = rawdata['issue_inst']
    title = rawdata['title']
    
    email = []
    if site == "WOS":
        email.extend(rawdata['emails'].replace(" ", "").split(";"))
        
    elif site == "SCOPUS":
        emails = rawdata['correspondence_address'].split(" ")
        for i in emails:
            if "@" in i:
                email.append(i)

    return coauthor, year, keyword, journal, conference, title, email

def Secondary_filter2(name1, name2, site1, inst1, raw_one1, site2, inst2, raw_one2):
    inst_sim = 0
    weight = 0
    joc = 0
    email_exact = 0
    coauthor1, year1, keyword1, journal1, conference1, title1, emails1 = filter(raw_one1, site1)
    coauthor2, year2, keyword2, journal2, conference2, title2, emails2 = filter(raw_one2, site2)
    
    for email1 in emails1:
        for email2 in emails2:
            if email1 == email2:
                email_exact = 1
                # print('-----------------이메일 통합-----------------')

    if inst1 == inst2:
        inst_sim = 1
    else:
        inst_sim = jaro.jaro_winkler_metric(inst1, inst2)

    if name1 in coauthor1:
        coauthor1.remove(name1)

    if name2 in coauthor2:
        coauthor2.remove(name2)
    
    co_author_count = len([i for i in coauthor1 if i in coauthor2])
    
    paper_sim = jaro.jaro_winkler_metric(title1.lower(), title2.lower())
    # if paper_sim >= 0.8:
    #     print('-----------------논문 통합-----------------')
    # if  inst_sim >= 0.8:
    #     print('-----------------소속 통합-----------------')
        
        
    if paper_sim >= 0.8 or inst_sim >= 0.8 or email_exact == 1:
        weight = 4
        # print(f'weigth: {weight} | joc: {joc} | yop: {year1}, {year2} | co_author_count: {co_author_count}| keyword: {keyword1}, {keyword2}')
        # print(f'name1: {name1} | inst1: {inst1} | title1: {title1} | site1: {site1}')
        # print(f'name2: {name2} | inst2: {inst2} | title2: {title2} | site1: {site2}')
        # print(f'inst: {inst_sim} | journal1: {journal1} | journal2: {journal2}')
        # print(f'email: {email_exact}')
        # print('------------------------------------------------------------구분선2------------------------------------------------------------')
        return weight

    else:
        joc = 1 if journal1 == journal2 and conference1 == conference2 else 0

            
    yop = -(2*(abs(year1-year2)/20)-1)
            
    if len(coauthor1) == 0 or len(coauthor2) == 0:
        co_author_ratio = 0
    elif len(coauthor1) < len(coauthor2):
        co_author_ratio = co_author_count/len(coauthor1)
    else:
        co_author_ratio = co_author_count/len(coauthor2)
    
    if co_author_ratio == 1:
        co_authorship = 1
    else:
        co_authorship = (1 - np.exp(-co_author_count))/2 + (co_author_ratio/2)
        
    keyword = 1 - np.exp(-len([i for i in keyword1 if i in keyword2]))

    weight = joc + yop + co_authorship + keyword
    # print(f'weigth: {weight} | joc: {joc} | yop: {yop} | co_authorship: {co_authorship} | keyword: {keyword}')
    # print(f'name1: {name1} | site1: {site1} | year: {year1} | coauthor: {coauthor1} | keyword: {keyword1} | journal1: {journal1}')
    # print(f'name2: {name2} | site1: {site2} | year: {year2} | coauthor: {coauthor2} | keyword: {keyword2} | journal2: {journal2}')
    # print(f'paper_sim: {paper_sim}')
    # print(f'title1: {title1}')
    # print(f'title2: {title2}')
    # print(f'inst_sim: {inst_sim}')
    # print(f'inst1: {inst1}')
    # print(f'inst2: {inst2}')
    # print(f'email_exact: {email_exact}')
    # print(f'emails1: {emails1}')
    # print(f'emails2: {emails2}')
    # # print(raw_one1)
    # # print(raw_one2)
    # print('------------------------------------------------------------구분선------------------------------------------------------------')
    
    return weight

In [5]:
raw_dbs = {'WOS' : wos_raw, 'SCOPUS' : scopus_raw}
savetime1 = 0
savetime2 = 0
def getRaw(name):
    if 'raws' not in Answer_dict[name]:
        raws = []
        for site_one in site:
            if site_one in Answer_dict[name]:
                for c in raw_dbs[site_one].find({"_id": {"$in": Answer_dict[name][site_one]['papers']}}):
                    c['site'] = site_one
                    raws.append(c)
        Answer_dict[name]['raws'] = raws

processedList = []
deleteList = []
count_rule = 0

for Answer_one in Answer_dict:
    if '_' in Answer_one :
        start1 = time.time()
        name = Answer_one.split("_")
        if name[0] in processedList :
            continue
        preprocessedList = []
        c = 0
        while True :
            pname = name[0]+"_"+str(c)
            if pname in Answer_dict :            
                preprocessedList.append(pname)
                getRaw(pname)
                c += 1
            else :
                break
        end1 = time.time()
        savetime1 += end1 - start1
        processedList.append(name[0])
        flag = True
        while flag :
            flag = False
            pairs =list(itertools.combinations(preprocessedList, 2))
            for pair in pairs:
                pair = list(pair)
                raws1 = Answer_dict[pair[0]]['raws']
                raws2 = Answer_dict[pair[1]]['raws']
                for ra1, ra2 in zip(raws1, raws2):
                    site1 = ra1['site']
                    site2 = ra2['site']
                    inst1 = Answer_dict[pair[0]][site1]['oriInst']
                    inst2 = Answer_dict[pair[1]][site2]['oriInst']
                    count_rule += 1
                    real_name1 = Answer_dict[pair[0]]['name']
                    real_name2 = Answer_dict[pair[1]]['name']
                    # print(f'input: {real_name1}, {real_name2}')
                    if Secondary_filter2(real_name1, real_name2, site1, inst1, ra1, site2, inst2, ra2) >= 3:
                        Inte_name.append(pair[0])
                        deleteList.append(pair[1])
                        for site_one in site:
                            if site_one in Answer_dict[pair[1]]:
                                if site_one in Answer_dict[pair[0]].keys() :                            
                                    Answer_dict[pair[0]][site_one]['A_id'].extend(Answer_dict[pair[1]][site_one]['A_id'])
                                    Answer_dict[pair[0]][site_one]['papers'].extend(Answer_dict[pair[1]][site_one]['papers'])
                                    Answer_dict[pair[0]]['raws'].extend(Answer_dict[pair[1]]['raws'])                        
                                    Answer_dict[pair[0]][site_one]['A_id'] = list(set(Answer_dict[pair[0]][site_one]['A_id']))
                                    Answer_dict[pair[0]][site_one]['papers'] = list(set(Answer_dict[pair[0]][site_one]['papers']))
                                else:
                                    Answer_dict[pair[0]][site_one] = Answer_dict[pair[1]][site_one]
                                if Answer_dict[pair[0]]['inst'] == "" or Answer_dict[pair[0]]['inst'] == " ":
                                    Answer_dict[pair[0]]['inst'] = Answer_dict[pair[0]][site_one]['oriInst']
                        flag = True
                        preprocessedList.remove(pair[1])
                        break
                if flag :
                    break

print(deleteList)

for del_name in deleteList:
    if del_name in Answer_dict:
        del Answer_dict[del_name]
for del_raw in Answer_dict : 
    if 'raws' in Answer_dict[del_raw] :
        del Answer_dict[del_raw]['raws']

['ungprasertpatompong_1', 'ungprasertpatompong_2', 'ungprasertpatompong_3', 'ungprasertpatompong_4', 'ungprasertpatompong_6', 'xiangyutao_1', 'xiangyutao_2', 'xiangyutao_4', 'xiangyutao_5', 'adlerdouglasg_1', 'adlerdouglasg_2', 'asemizatollah_1', 'asemizatollah_2', 'asemizatollah_3', 'asemizatollah_4', 'feldmanstevenr_1', 'feldmanstevenr_2', 'feldmanstevenr_3', 'kaochiahung_1', 'kaochiahung_2', 'kaochiahung_3', 'kaochiahung_4', 'kaochiahung_5', 'kaochiahung_6', 'kaochiahung_7', 'kaochiahung_8', 'kaochiahung_9', 'kaochiahung_10', 'kaochiahung_11', 'kaochiahung_12', 'kaochiahung_13', 'kaochiahung_14', 'kaochiahung_15', 'kaochiahung_16', 'delendursun_2', 'karakiewiczpierrei_2', 'karakiewiczpierrei_4', 'karakiewiczpierrei_5', 'yasunagahideo_1', 'yasunagahideo_2', 'yasunagahideo_4', 'yasunagahideo_5', 'yasunagahideo_6', 'yasunagahideo_8', 'yasunagahideo_9', 'yasunagahideo_10', 'yasunagahideo_11', 'yasunagahideo_12', 'yasunagahideo_13', 'yasunagahideo_14', 'yasunagahideo_15', 'yasunagahideo_

In [6]:
deleteList = []
sameLen_list = {}

for Answer_one in Answer_dict:
    for Answer_two in Answer_dict:
        name_sim = jaro.jaro_winkler_metric(Answer_one, Answer_two)
        if name_sim >= 0.8 and Answer_one != Answer_two:
            getRaw(Answer_one)
            getRaw(Answer_two)

            if len(Answer_one) < len(Answer_two):
                temp = Answer_one
                Answer_one = Answer_two
                Answer_two = temp
            elif len(Answer_one) == len(Answer_two):                
                if '_' in Answer_one and '_' in Answer_two:
                    numSame1 = Answer_one.split("_")[1]
                    numSame2 = Answer_two.split("_")[1]
                    if numSame1 > numSame2:
                        temp = Answer_one
                        Answer_one = Answer_two
                        Answer_two = temp

                else:
                    sameLen_list[Answer_one] = Answer_two
                    if Answer_two in sameLen_list: 
                        if Answer_one in sameLen_list[Answer_two]:
                            continue
            
            raws1 = Answer_dict[Answer_one]['raws']
            raws2 = Answer_dict[Answer_two]['raws']
            for ra1, ra2 in zip(raws1, raws2):
                site1 = ra1['site']
                site2 = ra2['site']
                inst1 = Answer_dict[Answer_one][site1]['oriInst']
                inst2 = Answer_dict[Answer_two][site2]['oriInst']
                count_rule += 1
                real_name1 = Answer_dict[Answer_one]['name']
                real_name2 = Answer_dict[Answer_two]['name']
                # print(f'input: {real_name1}, {real_name2}')
                # if Secondary_filter2(real_name1, real_name2, site1, inst1, ra1, site2, inst2, ra2) >= 3:
                if Secondary_filter2(real_name1, real_name2, site1, inst1, ra1, site2, inst2, ra2) >= 3:
                    Inte_name.append(Answer_one)
                    deleteList.append(Answer_two)
                    for site_one in site:
                        if site_one in Answer_dict[Answer_two]:
                            
                            if site_one in Answer_dict[Answer_one].keys() :                            
                                Answer_dict[Answer_one][site_one]['A_id'].extend(Answer_dict[Answer_two][site_one]['A_id'])
                                Answer_dict[Answer_one][site_one]['papers'].extend(Answer_dict[Answer_two][site_one]['papers'])
                                Answer_dict[Answer_one]['raws'].extend(Answer_dict[Answer_two]['raws'])
                                Answer_dict[Answer_one][site_one]['A_id'] = list(set(Answer_dict[Answer_one][site_one]['A_id']))
                                Answer_dict[Answer_one][site_one]['papers'] = list(set(Answer_dict[Answer_one][site_one]['papers']))
                            else:
                                Answer_dict[Answer_one][site_one] = Answer_dict[Answer_two][site_one]
                            if Answer_dict[Answer_one]['inst'] == "" or Answer_dict[Answer_one]['inst'] == " ":
                                Answer_dict[Answer_one]['inst'] = Answer_dict[Answer_one][site_one]['oriInst']
                    flag = True
                    break
            if flag :
                break
            
print(deleteList)

for del_name in deleteList:
    if del_name in Answer_dict:
        del Answer_dict[del_name]
for del_raw in Answer_dict : 
    if 'raws' in Answer_dict[del_raw] :
        del Answer_dict[del_raw]['raws']
        
print(len(Inte_name))
print(count_rule)

['fallond', 'katmawisabbaghs', 'ubertibonam', 'zoorobrj', 'stjohnmaiea', 'mossil', 'yaghoobzadeha', 'fingertjh', 'merajp', 'ceballosantonio', 'lapidothm', 'ottom', 'oginoh', 'yehgy', 'nananralph', 'tundian', 'meisterkm', 'musholttj', 'siskinddan', 'dekovenm', 'dawsonna', 'borgströmf', 'deinsbergerw', 'wattb', 'xiangyutao_3', 'gullapallin', 'sentelhaspauloc', 'lynchbm', 'afifyha', 'pitiakoudismichail', 'onakpoyaigho', 'lyrajrdivaldopereira', 'kakhkimm', 'lippig', 'bilenma', 'craftelena', 'lundgrenlena', 'lohofffw', 'chongtt', 'espregueiramendesj', 'tsukaharary', 'uusitaloh', 'higginsjm', 'allegaertk', 'nugentl', 'semciwa', 'kaffenbergerbenjamin', 'kropotovj', 'wittsteinjocelynr', 'pillayr', 'mungcharoent', 'viettov', 'greeneric', 'muhuripranabk', 'stonegreggw', 'temelkovakurktschievt', 'perezacostaandresm', 'ślaskabrygida', 'mittendorfea', 'caiz', 'paknahadz', 'puigcerverarandap', 'illicalem', 'başerda', 'ottp', 'oyibose', 'sylvesters', 'degeeh', 'kostevk_1', 'vreugdenhilace', 'kleinjt'

In [149]:
import json

answer = None
numCor = 0
numErr = 0
numInteErr = 0
answerCounter = {}

def ansCheck(result, name):
    global answer, numCor, numErr, answerCounter, numInteErr # answer = json, numCor = 정답 수, numErr = 오답 수, answerCounter = 정답수, numInteErr = 통합에러
    flag = True

    numK = len(result.keys()) #Answer_dict key 개수
    if numK-5 != len(answer[name].keys()):
        flag = False
    else :
        for site in result.keys():
            if site != 'name' and site != 'fid' and site != 'keyId' and site != 'inst' and site != 'pre_name':
                if site not in answer[name].keys() :
                    flag = False
                else: 
                    if numK > 5:
                        if answer[name][site] != result[site]['inst']:
                            flag = False
                    else :
                        if answer[name][site] != result[site]['inst'] and answer[name][site] != result[site]['inst'].split(' ')[0]:
                            flag = False
    if flag:
        numCor += 1
        answerCounter[name] += 1
    else:
        if len(answer[name].keys()) > 1:
            numInteErr += 1
            print(f"IIIIIIIIIIIIIIIIIIIIIIIIIIII Not Correct \n - Answer : {name} {answer[name]}\n - Result : {result}")
        else:
            pass
        numErr += 1
    return flag

with open(f'answer800.json', 'r',encoding='utf-8-sig') as a_json:

    answer = json.load(a_json)
    numAns = print(len(answer))
    for key in answer.keys() :
        answerCounter[key] = 0

    for answer_one in Answer_dict:        
        name = Answer_dict[answer_one]['name'].split('_')[0]

        if name in answer :
            ansCheck(Answer_dict[answer_one], name)

        else :
            count = 0
            while name+'_'+str(count) in answer :
                if ansCheck(Answer_dict[answer_one], name+'_'+str(count)) :
                    break
                count += 1

    print(f"num코렉트 {numCor}")
    print(f"numEror {numErr}")
    print(f"numInteEror {numInteErr}")
    totalError = 0
    totalInteError1 = 0
    totalInteError2 = 0
    answercount = 0
    totalInte = 0

    for key in answerCounter.keys() :
        answercount += 1
        if answerCounter[key] == 0 :
            totalError += 1
            print("answer=",key, answer[key])
            if len(answer[key].keys()) > 1 :
                totalInteError1 += 1

        else:
            if len(answer[key].keys()) > 1 :
                totalInteError2 += 1

    print("토탈앤써", answercount)
    print("토탈인터그레이션", totalInteError1 + totalInteError2)
    print("--------------------------------")
    print("토탈에러", totalError) 
    print("토탈인터그레이션에러", totalInteError1)
    print(keyid)
    
print(sorted(Answer_dict.items()))


1237
IIIIIIIIIIIIIIIIIIIIIIIIIIII Not Correct 
 - Answer : Pakzad, Shamim N. {'SCOPUS': 'Department of Civil and Environmental Engineering Lehigh University ATLSS Engineering Research Center 117 ATLSS Drive Imbt Labs Bethlehem PA  18015 United States', 'WOS': 'Lehigh Univ, Dept Civil & Environm Engn, 117 ATLSS Dr, Bethlehem, PA 18015 USA'}
 - Result : {'fid': 0, 'keyId': 884, 'name': 'Pakzad, Shamim N.', 'pre_name': 'pakzadshamimn', 'inst': 'Lehigh Univ, Dept Civil & Environm Engn, 117 ATLSS Dr, Bethlehem, PA 18015 USA', 'WOS': {'inst': 'Lehigh Univ, Dept Civil & Environm Engn, 117 ATLSS Dr, Bethlehem, PA 18015 USA', 'A_id': ['s2293137', 's6894904'], 'papers': [ObjectId('628decc859f4cc59911beeeb'), ObjectId('628decc859f4cc59911bee45')], 'oriInst': 'Lehigh Univ, Dept Civil & Environm Engn, 117 ATLSS Dr, Bethlehem, PA 18015 USA'}}
IIIIIIIIIIIIIIIIIIIIIIIIIIII Not Correct 
 - Answer : Chen, Wei {'SCOPUS': 'Innovative Institute of Chinese Medicine and Pharmacy Chengdu University of Traditi

In [42]:
pair_test = ['biswasrajib', 'biswasr']
  
getRaw(pair_test[0])
getRaw(pair_test[1])

raws1 = Answer_dict[pair_test[0]]['raws']
raws2 = Answer_dict[pair_test[1]]['raws']

name1 = Answer_dict[pair_test[0]]['name']
name2 = Answer_dict[pair_test[1]]['name']

for ra1, ra2 in zip(raws1, raws2):
    site1 = ra1['site']
    site2 = ra2['site']
    inst1 = Answer_dict[pair_test[0]][site1]['oriInst']
    inst2 = Answer_dict[pair_test[1]][site2]['oriInst']
    Secondary_filter2(name1, name2, site1, inst1, ra1, site2, inst2, ra2)

weigth: 4 | joc: 0 | yop: 2021, 2021 | co_author_count: 0| keyword: [['T']], [['T']]
name: Biswas, Rajib | inst1: Department of Physics Tezpur University Tezpur 784028 India | title1: Outlining Big Data Analytics in Health Sector with Special Reference to Covid-19 | site1: SCOPUS
name: Biswas, R. | inst2: Tezpur Univ, Dept Phys, Tezpur 784028, Assam, India | title2: Outlining Big Data Analytics in Health Sector with Special Reference to Covid-19 | site1: WOS
inst: 0.7179874517860559 | journal1: Springer | journal2: WIRELESS PERS COMMUN
------------------------------------------------------------구분선2------------------------------------------------------------


In [55]:
pair_test = ['shoshaniarie', 'shoshania'] 
pair_test = ['gagliardopablo', 'gagliardop'] 
pair_test = ['huangqian', 'huangq'] 
pair_test = ['venegasmartinezfrancisco', 'venegasmartínezf'] 
pair_test = ['alzahranifahad', 'alzahranifahda']
pair_test = ['sim´eonjérôme', 'simeonjerome']
pair_test = ['grothpault', 'grothpaul']
pair_test = ['szymańskił', 'szymanskil']
pair_test = ['burinskienėaurelija', 'burinskieneaurelija']
pair_test = ['herrerafrancisco_0', 'herrerafrancisco_1'] 
pair_test = ['pakzadshamimn_0', 'pakzadshamimn_2'] 
pair_test = ['chungyehching_0', 'chungyehching_1']
pair_test = ['huangmaolin_0', 'huangmaolin_1']
pair_test = ['dhavachelvanp_0', 'dhavachelvanp_1']
pair_test = ['bajwahassan_0', 'bajwahassan_1']
pair_test = ['gugulothunarsimha_0', 'gugulothunarsimha_1']
pair_test = ['kimjeongjoon_0', 'kimjeongjoon_1']
pair_test = ['elsayedayman_0', 'elsayedayman_1']
pair_test = ['heshunmin_0', 'heshunmin_1']
pair_test = ['arunachalamar_0', 'arunachalamar_1']
pair_test = ['sharmasanjiv_0', 'sharmasanjiv_1']
pair_test = ['bouamranekarim_0', 'bouamranekarim_2']
pair_test = ['jeyanthin_0', 'jeyanthin_2']
pair_test = ['liguoming', 'ligm']
pair_test = ['chiucarter', 'chiuc'] 
pair_test = ['kwonjiwoong', 'kwonjw'] 
pair_test = ['stepanovalexanderv', 'stepanovav'] 
pair_test = ['balasubramanianmahendran', 'balasubramanianm'] 
pair_test = ['wittebolexavier', 'wittebolex'] 
pair_test = ['nikolaevichtisenkovictor', 'nikolaevichtv'] 
pair_test = ['elsoudanimagdyms', 'elsoudanimms'] 
pair_test = ['chenwei_0', 'chenwei_2']
pair_test = ['sokolenkoliudmyla', 'sokolenkol']
pair_test = ['igarashiyasuhiko', 'igarashiy']
pair_test = ['youkhyun', 'youkh']
pair_test = ['tomásjesúsgarcía', 'tomasjesus']
pair_test = ['rajcpethuru', 'rajpethuru']
pair_test = ['cristeavalentingabriel', 'cristeavalentin']
pair_test = ['sáfilipe', 'safilipe']
pair_test = ['gräbelstefan', 'graebelstefan']
pair_test = ['ionescusorincristian', 'ionescusorin']
pair_test = ['kaliappanm', 'kahappanm']
pair_test = ['jadiamrmohsen', 'jadiamr']
pair_test = ['rashidmohammadanmir', 'rashidmohammeda']
pair_test = ['biswasrajib', 'biswasr']

